# Biblioteki Pythona w analizie danych

## Tomasz Rodak

Wykład 6

---

Literatura:

- [PRML](https://www.microsoft.com/en-us/research/uploads/prod/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf) Christopher M. Bishop, "Pattern Recognition and Machine Learning", 2006.
- [PML-1](https://probml.github.io/pml-book/) Kevin P. Murphy, "Probabilistic Machine Learning: An Introduction", 2022.
- C.M. Bishop, [Deep Learning, Foundations and Concepts](https://www.bishopbook.com/)
- [Dokumentacja NumPy](https://numpy.org/doc/stable/)
- [Dokumentacja scikit-learn](https://scikit-learn.org/stable/)
- [Dokumentacja PyTorch](https://pytorch.org/docs/stable/index.html)

# Metoda gradientu prostego. Tensory i różniczkowanie automatyczne w PyTorch

## Optymalizacja metodą gradientu prostego

Załóżmy, że $L(\mathbf{w})$ jest funkcją straty dla modelu z parametrami $\mathbf{w}$. Funkcja $L$ zwraca nieujemną wartość reprezentującą stopień w jakim przewidywania modelu różnią się od rzeczywistych wartości. Typowe przykłady to:

**Błąd średniokwadratowy** (MSE) dla problemów regresji:
  
  \begin{equation*}
    L(\mathbf{w}) = \frac{1}{N} \sum_{i=1}^N (y_i - f(\mathbf{x}_i, \mathbf{w}))^2
  \end{equation*}

gdzie $y_i$ to rzeczywista wartość, a $f(\mathbf{x}_i, \mathbf{w})$ to przewidywana wartość przez model dla danych wejściowych $\mathbf{x}_i$.

**Entropia krzyżowa** (*cross-entropy*) dla problemów klasyfikacji binarnej:

  \begin{equation*}
    L(\mathbf{w}) = -\frac{1}{N} \sum_{i=1}^N \left( y_i \log(f(\mathbf{x}_i, \mathbf{w})) + (1 - y_i) \log(1 - f(\mathbf{x}_i, \mathbf{w})) \right)
  \end{equation*}

gdzie $y_i$ to etykieta klasy (0 lub 1), a $f(\mathbf{x}_i, \mathbf{w})$ to przewidywana prawdopodobieństwo przynależności do klasy 1.

**Entropia krzyżowa** (*cross-entropy*) dla problemów klasyfikacji wieloklasowej:

  \begin{equation*}
    L(\mathbf{w}) = -\frac{1}{N} \sum_{i=1}^N \sum_{j=1}^K y_{ij} \log(f_j(\mathbf{x}_i, \mathbf{w}))
  \end{equation*}

gdzie $y_{ij}$ to etykieta klasy (1 jeśli próbka $i$ należy do klasy $j$, 0 w przeciwnym razie), a $f_j(\mathbf{x}_i, \mathbf{w})$ to przewidywana prawdopodobieństwo przynależności do klasy $j$.

Trening modelu polega na minimalizacji funkcji straty $L(\mathbf{w})$ względem parametrów $\mathbf{w}$. Metoda wykorzystywana do znalezienia minimum lokalnego funkcji $L(\mathbf{w})$ zależy przede wszystkim od tego, czy funkcja $L$ jest różniczkowalna względem $\mathbf{w}$, czy też nie:
- **Jeśli $L(\mathbf{w})$ jest różniczkowalna**, to zwykle korzysta się z jakiegoś wariantu metody gradientu prostego (o czym opowiemy za chwilę).
- **Jeśli $L(\mathbf{w})$ nie jest różniczkowalna**, lub informacja o gradientach nie jest dostępna, to stosuje się metody takie jak algorytmy ewolucyjne, optymalizację bayesowską, symulowane wyżarzanie, i wiele innych. 

Optymalizacja metodą gradientu prostego jest algorytmem iteracyjnym. Polega on na tym, że w pierwszym kroku wybieramy jakąś wartość początkową dla parametrów $\mathbf{w}^{(0)}$ (np. losowo), a następnie w każdym kroku $\tau$ aktualizujemy parametry $\mathbf{w}^{(\tau)}$ zgodnie z równaniem:

\begin{equation*}
  \mathbf{w}^{(\tau)} = \mathbf{w}^{(\tau - 1)} + \left(\text{krok zależny od }\mathbf{w}^{(\tau - 1)}\right)
\end{equation*}

Wybór kroku zależnego od $\mathbf{w}^{(\tau - 1)}$ jest kluczowy dla działania algorytmu i właśnie w tym miejscu wykorzystuje się informację o gradiencie funkcji $L(\mathbf{w})$ w punkcie $\mathbf{w}^{(\tau - 1)}$.

### *Batch gradient descent*

Termin *batch* oznacza, że funkcję straty $L(\mathbf{w})$ będziemy obliczali na podstawie całego zbioru danych. Matematycznie *batch gradient descent* definiuje ciąg $\mathbf{w}^{(\tau)}$ wzorem indukcyjnym:

\begin{equation*}
\begin{aligned}
  \mathbf{w}^{(0)} &= \text{losowo ustalona początkowa wartość parametrów modelu} \\
  \mathbf{w}^{(\tau)} &= \mathbf{w}^{(\tau - 1)} - \eta \nabla_{\mathbf{w}} L(\mathbf{w}^{(\tau - 1)}),\quad \tau = 1, 2, \ldots
\end{aligned}
\end{equation*}

gdzie **współczynnik uczenia** (*learning rate*) $\eta>0$ jest hiperparametrem algorytmu.

*Batch gradient descent* w pseudokodzie:

> **Input**: 
>   * zbiór danych $\mathcal{D} = \{(\mathbf{x}_n, y_n)\}_{n=1}^N$,
>   * funkcja straty $L(\mathbf{w})$ obliczana na podstawie całego zbioru danych $\mathcal{D}$
>   * współczynnik uczenia $\eta>0$,
>   * losowo inicjalizowana początkowa wartość parametrów modelu $\mathbf{w}$.
> 
> **Output**: 
>   * parametry modelu $\mathbf{w}$ po zakończeniu treningu.
>
> **Algorytm**:
>
> 1. Powtarzaj:
>    * $\mathbf{w} \gets \mathbf{w} - \eta \nabla_{\mathbf{w}} L(\mathbf{w})$.
> 2. aż do osiągnięcia zbieżności.
> 3. Zwróć $\mathbf{w}$.

### *Stochastic gradient descent* (SGD)

Funkcja straty $L(\mathbf{w})$ często daje się zapisać jako suma strat dla poszczególnych próbek:

\begin{equation*}
  L(\mathbf{w}) = \frac{1}{N} \sum_{n=1}^N L_n(\mathbf{w}),
\end{equation*}

gdzie $L_n(\mathbf{w})$ to strata dla próbki $n$. Przykładowo, dla problemu regresji z funkcją straty MSE mamy:

\begin{equation*}
  L(\mathbf{w}) = \frac{1}{N} \sum_{n=1}^N (y_n - f(\mathbf{x}_n, \mathbf{w}))^2 = \frac{1}{N} \sum_{n=1}^N L_n(\mathbf{w}),
\end{equation*}

gdzie $L_n(\mathbf{w}) = (y_n - f(\mathbf{x}_n, \mathbf{w}))^2$. Podobnie dla entropii krzyżowej mamy:

\begin{equation*}
  L(\mathbf{w}) = -\frac{1}{N} \sum_{n=1}^N\sum_{j=1}^K y_{nj} \log(f_j(\mathbf{x}_n, \mathbf{w})) = \frac{1}{N} \sum_{n=1}^N L_n(\mathbf{w}),
\end{equation*}

gdzie $L_n(\mathbf{w}) = -\sum_{j=1}^K y_{nj} \log(f_j(\mathbf{x}_n, \mathbf{w}))$. 

Jeśli $L(\mathbf{w})$ można zapisać jak wyżej, to gradient $\nabla_{\mathbf{w}} L(\mathbf{w})$ jest równy średniej z gradientów $L_n(\mathbf{w})$:

\begin{equation*}
  \nabla_{\mathbf{w}} L(\mathbf{w}) = \frac{1}{N} \sum_{n=1}^N \nabla_{\mathbf{w}} L_n(\mathbf{w}).
\end{equation*}

Obserwacja ta jest wykorzystywana w algorytmie *stochastic gradient descent* (SGD), który polega na tym, że w każdym kroku aktualizujemy parametry $\mathbf{w}^{(\tau)}$ wykorzystując gradient tylko dla jednej próbki $n$ zamiast dla całego zbioru jak w przypadku *batch gradient descent*. Matematycznie SGD definiuje ciąg $\mathbf{w}^{(\tau)}$ wzorem indukcyjnym:

\begin{equation*}
\begin{aligned}
  \mathbf{w}^{(0)} &= \text{losowo ustalona początkowa wartość parametrów modelu} \\
  \mathbf{w}^{(\tau)} &= \mathbf{w}^{(\tau - 1)} - \eta \nabla_{\mathbf{w}} L_n(\mathbf{w}^{(\tau - 1)}),\quad \tau = 1, 2, \ldots
\end{aligned}
\end{equation*}
gdzie $n$ jest numerem próbki zależnym od kroku $\tau$.

SGD w pseudokodzie:

> **Input**:
>   * zbiór danych $\mathcal{D} = \{(\mathbf{x}_n, y_n)\}_{n=1}^N$,
>   * funkcja straty $L_n(\mathbf{w})$ obliczana na podstawie próbki $n$,
>   * współczynnik uczenia $\eta>0$,
>   * losowo inicjalizowana początkowa wartość parametrów modelu $\mathbf{w}$.
>
> **Output**:
>   * parametry modelu $\mathbf{w}$ po zakończeniu treningu.
>
> **Algorytm**:
>
> 1. $n \gets 1$.
> 2. Powtarzaj:
>    * $\mathbf{w} \gets \mathbf{w} - \eta \nabla_{\mathbf{w}} L_n(\mathbf{w})$
>    * $n \gets n + 1$
>    * jeśli $n > N$, to:
>      * wymieszaj dane $\mathcal{D}$,
>      * $n \gets 1$.
> 3. aż do osiągnięcia zbieżności.
> 4. Zwróć $\mathbf{w}$.

Algorytm ten w ciągu $N$ kroków pętli wykona $N$ aktualizacji parametrów $\mathbf{w}$ przepuszczając przez model wszystkie próbki. Korzysta się z następującej terminologii:
* **iteracja** (*iteration*) - pojedyncza aktualizacja parametrów $\mathbf{w}$, czyli przetworzenie jednej próbki,
* **epoka** (*epoch*) - pełne przejście przez cały zbiór danych, czyli $N$ iteracji.

### *Mini-batch gradient descent*

*Mini-batch gradient descent* jest kompromisem pomiędzy *batch gradient descent* a SGD: aktualizacja parametrów $\mathbf{w}^{(\tau)}$ odbywa się na podstawie gradientu funkcji straty $L(\mathbf{w})$ obliczonego na podstawie *mini-batcha* próbek - podzbioru większego niż jedna próbka, ale mniejszego (zwykle znacznie) niż cały zbiór. 

Niech $B$ będzie liczbą próbek w *mini-batchu*. Jest to, podobnie jak $\eta$, hiperparametr algorytmu. Formalnie $B$ jest dowolną liczbą między 1 a $N$, w praktyce jest zazwyczaj ustalane jako jakaś potęga 2. Możemy teraz dokonać uogólnienia wzoru z paragrafu o SGD:

\begin{equation*}
\begin{split}
  L(\mathbf{w}) &= \frac{1}{N} \sum_{n=1}^N L_n(\mathbf{w}) 
  = \frac{B}{N} \sum_{k=1}^{N/B} \frac{1}{B} \sum_{j=1}^B L_{(k-1)B + j}(\mathbf{w})\\
  &= \frac{B}{N} \sum_{k=1}^{N/B} L_{(k-1)B + 1, (k-1)B + 2, \ldots, kB}(\mathbf{w}),
\end{split}
\end{equation*}

gdzie $L_{(k-1)B + 1, (k-1)B + 2, \ldots, kB}(\mathbf{w})$ to strata dla próbek z *mini-batcha* o numerach od $(k-1)B + 1$ do $kB$. W każdym kroku aktualizujemy parametry $\mathbf{w}^{(\tau)}$ wykorzystując gradient obliczony na batchu $B$ próbek.

Matematycznie:

\begin{equation*}
\begin{aligned}
  \mathbf{w}^{(0)} &= \text{losowo ustalona początkowa wartość parametrów modelu} \\
  \mathbf{w}^{(\tau)} &= \mathbf{w}^{(\tau - 1)} - \eta \nabla_{\mathbf{w}} L_{n_1, n_2, \ldots, n_B}(\mathbf{w}^{(\tau - 1)}),\quad \tau = 1, 2, \ldots
\end{aligned}
\end{equation*}

gdzie $n_1, n_2, \ldots, n_B$ to numery próbek w *mini-batchu* zależne od kroku $\tau$.

*Mini-batch gradient descent* w pseudokodzie:

> **Input**:
>   * zbiór danych $\mathcal{D} = \{(\mathbf{x}_n, y_n)\}_{n=1}^N$,
>   * funkcja straty $L_{n:n+B}(\mathbf{w})$ obliczana na podstawie próbek $n:n+B$,
>   * współczynnik uczenia $\eta>0$,
>   * rozmiar *mini-batcha* $B$,
>   * losowo inicjalizowana początkowa wartość parametrów modelu $\mathbf{w}$.
>
> **Output**:
>   * parametry modelu $\mathbf{w}$ po zakończeniu treningu.
>
> **Algorytm**:
> 1. $n \gets 1$.
> 2. Powtarzaj:
>    * $\mathbf{w} \gets \mathbf{w} - \eta \nabla_{\mathbf{w}} L_{n:n+B}(\mathbf{w})$
>    * $n \gets n + B$
>    * jeśli $n > N$, to:
>      * wymieszaj dane $\mathcal{D}$,
>      * $n \gets 1$.
> 3. aż do osiągnięcia zbieżności.
> 4. Zwróć $\mathbf{w}$.

W tym algorytmie **iteracja** to przetworzenie *mini-batcha* $B$ próbek, a **epoka** to, tak jak poprzednio, pełne przejście przez cały zbiór danych, czyli $N/B$ iteracji. 

Wyżej założyliśmy dla uproszczenia, że $N$ jest podzielne przez $B$. Jeśli tak nie jest, to w ostatniej iteracji po prostu przetwarzamy wszystkie pozostałe próbki, które nie zmieściły się w pełnych *mini-batchach*. 

## Tensory i różniczkowanie automatyczne w PyTorch

**Tensor** w PyTorch to wielowymiarowa tablica wartości numerycznych, podobna do tablic NumPy, ale z dodatkowymi możliwościami:

- **Obsługa GPU**: Tensory mogą być przetwarzane na GPU.
- **Różniczkowanie automatyczne**: PyTorch automatycznie oblicza gradienty dla operacji na tensorach.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

import torch

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
X1 = np.random.rand(500, 2)
X2 = (np.random.rand(500, 2) + .7)
X = np.concatenate((X1, X2), axis=0)
y = np.concatenate((np.zeros(500), np.ones(500)), axis=0)
ax.scatter(X[:, 0], X[:, 1], c=y, s=10, cmap='coolwarm');

In [ ]:
reglog = LogisticRegression()
reglog.fit(X, y)
y_pred = reglog.predict(X)
confusion_matrix(y, y_pred)

In [ ]:
accuracy_score(y, y_pred)

In [ ]:
# Zamiana numpy array na tensor
X_tensor = torch.tensor(X, dtype=torch.float32).view(-1, 2)
X_tensor = torch.cat((torch.ones(X_tensor.shape[0], 1), X_tensor), dim=1)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# Przygotowanie loaderów danych
train_dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
batch_size = 32
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)


# Definiowanie modelu
def sigmoid(x, w):
    return 1 / (1 + torch.exp(-(w[0] + w[1] * x[:, 1] + w[2] * x[:, 2])))

# Definiowanie funkcji straty
def binary_cross_entropy(y_true, y_pred):
    y_pred = torch.clamp(y_pred, 1e-7, 1 - 1e-7)  # zapobieganie log(0)
    return -torch.mean(y_true * torch.log(y_pred) + (1 - y_true) * torch.log(1 - y_pred))

# Inicjalizacja parametrów
w = torch.tensor(np.random.rand(3), dtype=torch.float32, requires_grad=True)

eta = .1 # learning rate

# Trenowanie modelu metodą mini-batch gradient descent
for epoch in range(10):
    for X_batch, y_batch in train_loader:
        y_pred = sigmoid(X_batch, w)
        loss = binary_cross_entropy(y_batch, y_pred)
        loss.backward() # obliczanie gradientów
        
        with torch.no_grad():
            # aktualizacja wartości parametrów
            w.data = w.data - eta * w.grad
            # zerowanie gradientów
            w.grad.zero_()
            
    print(f"Epoch {epoch + 1}/{10}, Loss: {loss.item()}")

In [ ]:
y_pred = sigmoid(X_tensor, w)
y_pred = y_pred > 0.5
confusion_matrix(y, y_pred.detach().numpy())

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Zamiana numpy array na tensor
X_tensor = torch.tensor(X, dtype=torch.float32).view(-1, 2)
X_tensor = torch.cat((torch.ones(X_tensor.shape[0], 1), X_tensor), dim=1)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# Przygotowanie loaderów danych
train_dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
batch_size = 32
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# Definiowanie modelu przy użyciu torch.nn
class LogisticRegression(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
    
    def forward(self, x):
        out = self.linear(x)
        out = torch.sigmoid(out)  
        return out

# Inicjalizacja modelu
model = LogisticRegression(input_dim=3)

# Definiowanie funkcji straty i optymalizatora
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.SGD(model.parameters(), lr=0.1)  # Stochastic Gradient Descent

# Trenowanie modelu
num_epochs = 10
for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        # Forward pass
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        
        # Backward pass i optymalizacja
        optimizer.zero_grad()  # zerowanie gradientów
        loss.backward()        # obliczanie gradientów
        optimizer.step()       # aktualizacja parametrów
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

In [ ]:
y_pred = model(X_tensor)
y_pred = y_pred > 0.5
confusion_matrix(y, y_pred.detach().numpy())

In [ ]:
accuracy_score(y, y_pred.detach().numpy())